In [4]:
import pandas as pd
import os
#!pip install sklearn
import sklearn

In [2]:
cancer_df = pd.read_csv('/data/team2/top10data/cancer.csv')
cancer_df['PMID'] = cancer_df['PMID'].apply(lambda x: x.split(','))
exploded_df = cancer_df.explode('PMID')
exploded_df = exploded_df.drop_duplicates('PMID')


top_10_query_PMIDs = pd.read_csv('/data/team2/top_10_query_PMIDs_v2.csv').drop('Unnamed: 0', axis=1)
top_10_query_PMIDs['PMID'] = top_10_query_PMIDs['pmid'].astype(str)

# adding hasAbstract, hasStructuredAbstract, pubtype
merged_df = exploded_df.merge(top_10_query_PMIDs, how='left', on = 'PMID')

# adding one-hot encoded publication types
merged_df1 = merged_df.join(pd.get_dummies(merged_df.pubtype, prefix='pubtype'))

In [3]:
merged_df1.head()

,search_id,query_term,filters,sort_algorithm,result_count,page_num,PMID,click_data,date,user_id,...,"pubtype_['Journal Article', 'Validation Study']",pubtype_['Journal Article'],"pubtype_['Letter', 'Comment']","pubtype_['Letter', 'Randomized Controlled Trial']",pubtype_['Letter'],"pubtype_['Meta-Analysis', ""Research Support, Non-U.S. Gov't"", 'Systematic Review']","pubtype_['News', 'Comment']",pubtype_['News'],pubtype_['Published Erratum'],pubtype_['Retraction of Publication']
0,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,31761807,NoClicks,2022-03-16,Z392386,...,0,1,0,0,0,0,0,0,0,0
1,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,28244479,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,0
2,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,27741350,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,0
3,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,30713326,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,0
4,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,29949179,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,0


In [7]:
len(merged_df)

2142

In [5]:
cancer_df = pd.read_csv('/data/team2/top10data/cancer.csv')

cancer_df

,search_id,query_term,filters,sort_algorithm,result_count,page_num,PMID,click_data,date,user_id,filters_active,processed_query
0,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,"31761807,28244479,27741350,30713326,29949179,2...",NoClicks,2022-03-16,Z392386,False,cancer
1,5cr7r_f0i_6wyd0xw3ayza:93764b25ae99d315f410802...,cancer,None,date,4569599,1,"35289556,35289548,35289536,35289535,35289519,3...",NoClicks,2022-03-16,Z401427,False,cancer
2,_hyofx-kvkxkzrumj0tvoa:a99dca5593185c498b63a5e...,cancer,None,relevance,4569599,1,"31761807,28244479,27741350,30713326,29949179,2...","*result_click,28244479,2022-03-16T20:03:49,1,F...",2022-03-16,Z272979,False,cancer
3,_wbpom97rhxkmccuzyb26g:7463543d784aa59ca86359a...,cancer,None,relevance,4569599,1,"31761807,28244479,27741350,30713326,29949179,2...",NoClicks,2022-03-16,Z93392,False,cancer
4,aajrft_asz6yj4tjzrqkdq:26985a88de1e140c5bce6ac...,cancer,datesearch.y_5,relevance,1064948,1,"29860986,31761807,30445141,28154351,30713326,2...","*result_click,31761807,2022-03-16T20:44:35,1,F...",2022-03-16,Z292543,True,cancer
...,...,...,...,...,...,...,...,...,...,...,...,...
434,zh8ka-z38c5-lt2u_es5pa:a99dca5593185c498b63a5e...,cancer,None,relevance,4565187,1,"31761807,28244479,27741350,30713326,29949179,2...",NoClicks,2022-03-09,Z391060,False,cancer
435,zixesdd3x1a8dq8qf40noa:bce3586a1d3a3cd33201d84...,cancer,"pubt.review,lang.portuguese",relevance,801,1,"28614430,29160494,32354458,29538545,32667426,2...","*result_click,29538545,2022-03-09T09:04:47,1,F...",2022-03-09,Z141330,True,cancer
436,zvydnkesoc5_lxcvysq50g:c2ba8420ea8e9b67e985e4d...,cancer,None,relevance,4565187,1,"31761807,28244479,27741350,30713326,29949179,2...",NoClicks,2022-03-09,Z201014,False,cancer
437,zx-dmpxxkv1fk8ustboiia:6a1cd9549015db5492d2984...,cancer,simsearch2.ffrft,relevance,1431624,1,"18538731,29860986,28244479,28154351,30713326,2...",NoClicks,2022-03-09,Z381937,True,cancer


In [6]:
top_10_query_PMIDs = pd.read_csv('/data/team2/top_10_query_PMIDs_v2.csv').drop('Unnamed: 0', axis=1)
top_10_query_PMIDs

,pmid,journal,title,abstract,hasAbstract,hasStructuredAbstract,articleDate,journalPubDate,entrezDate,pubtype
0,35252802,Journal of biosafety and biosecurity,Recent developments and trends of automatic nu...,"Nucleic acid detection, widely used in clinica...",True,False,2022-02-28,NaN,2022-03-07,"['Journal Article', 'Review']"
1,35287656,BMC public health,Nutrition standards for the charitable food sy...,Food insecurity is associated with increased r...,True,False,2022-03-14,2022-03-14,2022-03-15,"['Journal Article', ""Research Support, Non-U.S..."
2,35232441,BMC medical education,Establishing CREATE: lessons learned in settin...,"[{'@Label': 'BACKGROUND', '@NlmCategory': 'BAC...",True,True,2022-03-01,2022-03-01,2022-03-02,['Journal Article']
3,32282054,Turk patoloji dergisi,Pulmonary Acantholytic Squamous Cell Carcinoma...,NaN,False,False,NaN,NaN,2020-04-14,['Journal Article']
4,33629745,Journal of cellular physiology,TRIM11 suppresses ferritinophagy and gemcitabi...,Gemcitabine is first-line chemotherapy for pan...,True,False,2021-02-25,NaN,2021-02-25,"['Journal Article', ""Research Support, Non-U.S..."
...,...,...,...,...,...,...,...,...,...,...
16814,34281775,Phytomedicine : international journal of phyto...,Identification of prenylated phenolics in mulb...,"[{'@Label': 'BACKGROUND', '@NlmCategory': 'BAC...",True,True,2021-07-03,NaN,2021-07-20,['Journal Article']
16815,33762716,British journal of cancer,Conventional versus reverse sequence of neoadj...,"[{'@Label': 'BACKGROUND', '#text': 'Preoperati...",True,True,2021-03-24,NaN,2021-03-25,"['Clinical Trial, Phase II', 'Journal Article'..."
16816,35239864,Revista do Instituto de Medicina Tropical de S...,Adult patients admitted to a tertiary hospital...,COVID-19 is a disease whose knowledge is still...,True,False,2022-02-25,NaN,2022-03-03,"['Journal Article', 'Observational Study']"
16817,35163783,International journal of molecular sciences,Theranostic Interpolation of Genomic Instabili...,Breast cancer is a diverse disease caused by m...,True,False,2022-02-07,2022-02-07,2022-02-15,"['Journal Article', 'Review']"
